TODO
jason briner - long term perspectives of greenland ice sheet position
- if current approx margin configuration is where it _generally_ is during interglacials then it would *make sense* that there are lots of overdeepenings just up-ice.
- Lindbäck, K., R. Pettersson, A. L. Hubbard, S. H. Doyle, D. van As, A. B. Mikkelsen, and A. A. Fitzpatrick. 2015. Subglacial water drainage, storage and piracy beneath the Greenland ice sheet. Geophysical Research Letters 42:7606–14.
see lindback references in ross et al., 2018
- Lindbäck, K., R. Petterson, S. H. Doyle, C. Helanow, P. Jansson, S. S. Kristensen, L. Stenseng, R. Forsberg, and A. L. Hubbard. 2014. High-resolution ice thickness and bed topography of a land-terminating section of the Greenland ice sheet. Earth System Science Data 6:331–38.
is there evidence of *preferential* erosion signature

lake massu
simon mudd, mikael attal - what are lakes? 
rivers **cannot** erode below base level.

TODO
send PN velocity figures and summary paragraph

In [2]:
from dask.distributed import Client, LocalCluster
import pandas as pd
# import itslive
import geopandas as gpd
# from shapely.geometry import Polygon
# from shapely import box
import matplotlib.pyplot as plt
# from matplotlib.collections import LineCollection
# from matplotlib.dates import date2num, DateFormatter, YearLocator
# import seaborn as sns
# import xrspatial as xrs
import numpy as np
import xarray as xr
from tqdm import tqdm
from IPython import display
import importlib
# import imagery
import utils
import velocity_helpers

In [ ]:
# _ = importlib.reload(utils)
# _ = importlib.reload(velocity_helpers)
# _ = importlib.reload(imagery)

In [6]:
cluster = LocalCluster()
client = Client(cluster)
print(client.dashboard_link)

http://127.0.0.1:8787/status


In [13]:
## read in lines, and lazily get velocity cubes, centrelines, and robust trends
lines = gpd.read_file('../data/streams_v2.geojson')
V = {}
failed = []
for row in tqdm(lines.sample(1).itertuples()):
    print(f'working on #{row.Index}')
    try:
        V[row.Index] = velocity_helpers.CentreLiner(
            geo=row.geometry,
            buff_dist=3_000,
            index=row.Index,
            filter_cube=True,
            get_annual_trends=True,
            get_annual_median=False,
            get_rgb=False)
        display.clear_output(wait=True)
    except Exception as e:
        failed.append((row.Index, e))
        print(f'#{row.Index} did not work because\n{e}')
        continue

In [35]:
# compute robust trends and export at the same time
for k, v in tqdm(V.items()):
    v.robust_trend['v_trend'].attrs = {'crs':3413,
                                       'buffer':3000,
                                       'ddt_range':('335d', '395d'),
                                       'mad_n':5,
                                       'date':str(pd.Timestamp.now()),
                                       'centreline':v.tidy_stream.wkt,
                                       'centreline_id':k}
    
    (v.robust_trend['v_trend']
     .chunk(dic(zip(v.robust_trend['v_trend'].dims,
                    v.robust_trend['v_trend'].shape)))
     .to_zarr(f'../results/intermediate/velocity/robust_annual_trends/id{k}.zarr'))


## other

In [ ]:
fig, axs = plt.subplots(figsize=[15,15],
                        nrows=5,
                        ncols=3,
                        sharex=True,
                        sharey=True)

for k_v, ax in zip(V.items(), axs.flat):
    k, v = k_v
    get_velocity.Plotters.rolling_median(v,
                                         ('335d','395d'),
                                         ax=ax,
                                         ddt_bars=False,
                                         **{'var': 'v',
                                            'col': 'cumul_dist',
                                            'window': '90d',
                                            'vals':[1_000, 5_000]
                                            })
    _point = utils.shapely_reprojector(v.point, 3413, 4326)
    
    ax.set_title(f'#{k}: {_point.y:.2f}N {_point.x:.2f}W')

axs.flat[-2].remove()
axs.flat[-1].remove()

In [ ]:
def centreline_theil_slopes(df, **kwargs):
    
    _var = kwargs.get('var', 'v')
    _x = kwargs.get('x', 'mid_date')
    _date_range = kwargs.get('date_range', (pd.Timestamp('1900-01-01'),
                                            pd.Timestamp.now()))
    _ddt_range = kwargs.get('ddt_range', ('0d', '30d'))
    _mad = kwargs.get('mad', 3)
    
    _df = df.loc[df[_x].between(*_date_range)]
    
    
    

In [ ]:
idx = ((ds['date_dt'] > pd.Timedelta('335d')) & ((ds['date_dt'] <= pd.Timedelta('395d')))).compute()

from scipy.stats.mstats import theilslopes
from scipy.stats import linregress

x = (ds.mid_date[idx] - ds.mid_date[idx].min()) / pd.Timedelta('365.25d')
y = ds['v'][idx,50,50].compute()

nan_idx = y.isnull()

x = x[~nan_idx]
y= y[~nan_idx]

theil_result = theilslopes(y=y, x=x)
lr_result = linregress(x=x, y=y)


X = np.arange(0,30)
theil_fit = (X * theil_result.slope) + theil_result.intercept
lr_fit = (X * lr_result.slope) + lr_result.intercept

fig, ax = plt.subplots()
ax.scatter(x,y)

ax.plot(X, theil_fit, c='r', label='theilslopes')
ax.plot(X, lr_fit, c='g', label='lr')
ax.legend()

In [ ]:
fig, axs = plt.subplots(figsize=[15,8], nrows=5, ncols=3)
# get_velocity.Plotters.date_dt_bars(V[13],
#                                    ('1d','15d'),
#                                    ax=ax,
#                                    **{'vals':[5_000,1_000]})

get_velocity.Plotters.rolling_median(V[13],
                                     ('335d','395d'),
                                     ax=ax,
                                     ddt_bars=False,
                                     **{'var': 'v',
                                        'col': 'cumul_dist',
                                        'window': '180d',
                                        'vals':[1_000, 3_000, 5_000, 10_000]})

- look at underlying dodgy velocity images
TODO look at GRIMpP and MEASUREs

TODO
- PN thinks that relative change is sufficient to *control* for variations in glacier geometry (read: width/size).
- most interested in *relative* change of velocity over final few kilometers.
    - i.e. 

- it all comes from the same raw processed v data

## imagery

In [ ]:
study_sites = gpd.read_file('../data/potential_study_sites_v1.geojson')
study_sites = study_sites.loc[study_sites['notes'].isin(['yes', 'maybe'])]
study_sites.reset_index(drop=True, inplace=True)
study_sites_5k = study_sites.buffer(5_000).to_crs(4326)

In [ ]:
items = imagery.get_annual_median_mosaic(
    study_sites.loc[0,'geometry'],
    buffer_dist=15_000,
    src_crs=study_sites.crs,
    target_crs=4326,
    timeperiod='2023-01-01/2024-01-01',
    months=[8,9],
    )

In [ ]:
(xrs.multispectral.true_color(
    *items.squeeze()
    .transpose('band', 'y', 'x'))
 .plot.imshow(rgb='band')
)

In [ ]:
import stackstac
stackstac.stack(items, assets=['B04', 'B03', 'B02'])